In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bappekim/air-pollution-in-seoul")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from scipy.spatial.distance import cdist

In [ ]:
BATCH_SIZE = 32
SEQ_LEN = 24
NUM_NODES = 13
NUM_FEATURES = 10
TARGET_DIM = 6 #New: changed to 6 from 1
LEARNING_RATE = 0.001
EPOCHS = 10

# New: Define exactly which columns we want to predict (indices 0 to 5)
TARGET_COLS = ['NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM2.5']
target_indices = [0, 1, 2, 3, 4, 5]

In [ ]:
def preprocess_seoul_data(df, scalers, look_back_window=24):
    # Ensure Date format
    df['Measurement date'] = pd.to_datetime(df['Measurement date'])

    # CRITICAL FIX: Sort by Date AND Station Code to ensure alignment
    df = df.sort_values(by=['Measurement date', 'Station code'])

    # Time Encoding
    df['hour'] = df['Measurement date'].dt.hour
    df['month'] = df['Measurement date'].dt.month
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24.0)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24.0)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12.0)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12.0)

    # Select Features
    feature_cols = ['NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM2.5',
                    'hour_sin', 'hour_cos', 'month_sin', 'month_cos']

    # Get sorted list of stations to ensure consistency
    stations = sorted(df['Station code'].unique())
    num_stations = len(stations)

    processed_features = []

    for feat in feature_cols:
        values = df[feat].values.reshape(-1, num_stations)

        # Interpolate missing values
        val_df = pd.DataFrame(values)
        val_df = val_df.interpolate(method='linear', limit_direction='both')
        # val_df = val_df.fillna(method='bfill').fillna(method='ffill') # Extra safety
        val_df = val_df.bfill().ffill()

        # Normalize only physical pollutants, not sin/cos
        if 'sin' not in feat and 'cos' not in feat:
            # scaler = MinMaxScaler()
            # values_norm = scaler.fit_transform(val_df.values)
            scaler_feat = MinMaxScaler()
            values_norm = scaler_feat.fit_transform(val_df.values)
            scalers[feat] = scaler_feat
        else:
            values_norm = val_df.values

        processed_features.append(values_norm)

    # Stack features: (Time, Nodes, Features)
    data_block = np.stack(processed_features, axis=-1)

    X, Y = [], []
    total_time_steps = data_block.shape[0]

    #new
    # target_idx = 5 # PM2.5
    # INDICES for: ['NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM2.5']
    target_indices = [0, 1, 2, 3, 4, 5]

    for i in range(total_time_steps - look_back_window - 1):
        X.append(data_block[i : i + look_back_window, :, :])
        # Y.append(data_block[i + look_back_window, :, target_idx])
        # CHANGED: Grab all target columns, not just index 5
        Y.append(data_block[i + look_back_window, :, target_indices])

    # return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(Y)).unsqueeze(-1)
    # CHANGED: Removed .unsqueeze(-1) because Y is already shape (Batch, Nodes, 6)
    return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(Y))

In [ ]:
df = pd.read_csv(path + '/AirPollutionSeoul/Measurement_summary.csv')
df_station = pd.read_csv(path + '/AirPollutionSeoul/Original Data/Measurement_station_info.csv')

# Filter Stations
keep_stations = [101, 102, 105, 106, 107, 109, 111, 112, 113, 119, 120, 121, 122]
df = df[df['Station code'].isin(keep_stations)]
df_station = df_station[df_station['Station code'].isin(keep_stations)]

# CRITICAL FIX: Sort station info so the graph matches the data tensors
df_station = df_station.sort_values('Station code')

# Generate Tensors
scalers = {}
train_x, train_y = preprocess_seoul_data(df, scalers, SEQ_LEN)
print(f"Data Prepared: Input {train_x.shape}, Target {train_y.shape}")

In [ ]:
import joblib
joblib.dump(scalers, "seoul_scalers.pkl")

In [ ]:
def get_adjacency_matrix_dynamic(lat_lon_df, threshold_percentile=0.5):
    """
    1. Calculates pairwise distances.
    2. Dynamically sets sigma based on the standard deviation of distances.
    3. Generates the Gaussian kernel graph.
    """
    # 1. Calculate Pairwise Distances (Euclidean for Lat/Lon)
    coords = lat_lon_df[['Latitude', 'Longitude']].values
    dists = cdist(coords, coords, metric='euclidean')

    # 2. Dynamic Sigma Calculation
    # We flatten the distance matrix and remove 0s (self-loops)
    non_zero_dists = dists[dists > 0]

    # Heuristic: Sigma = Standard Deviation of all distances
    # This automatically scales to the unit (Degrees vs Meters)
    sigma = np.std(non_zero_dists)

    print(f"--- Dynamic Graph Statistics ---")
    print(f"Calculated Sigma (Std Dev): {sigma:.6f}")

    # 3. Build Weight Matrix
    num_nodes = len(lat_lon_df)
    adj = np.zeros((num_nodes, num_nodes))

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                adj[i][j] = 1.0
            else:
                d = dists[i][j]
                # Gaussian Kernel
                weight = np.exp(- (d**2) / (sigma**2))
                adj[i][j] = weight

    # 4. Thresholding (Optional but Recommended)
    # Filter out weak connections to prevent the "Everything Connected" problem
    # We keep only edges stronger than the 'threshold_percentile' of weights
    # But usually, just checking if weight > epsilon (e.g. 0.1) is enough.

    weight_threshold = 0.1 # Edges weaker than this are cut
    adj[adj < weight_threshold] = 0.0

    # Check Connectivity
    num_edges = np.count_nonzero(adj) - num_nodes # subtract self-loops
    density = num_edges / (num_nodes * (num_nodes - 1))
    print(f"Graph Density: {density:.2%} (Target: 20-50%)")

    # 5. Normalization
    row_sum = np.sum(adj, axis=1)
    row_sum[row_sum == 0] = 1.0 # Prevent division by zero for isolated nodes
    adj_norm = adj / row_sum[:, np.newaxis]

    return torch.tensor(adj_norm, dtype=torch.float32)

In [ ]:
# adj_matrix = get_adjacency_matrix(df_station, sigma=0.06, threshold=0.0)
adj_matrix = get_adjacency_matrix_dynamic(df_station)

In [ ]:
torch.save(adj_matrix, "adj_matrix.pt")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_adjacency_matrix(adj_tensor, station_labels):
    """
    Plots the normalized adjacency matrix as a heatmap.
    """
    # Convert PyTorch Tensor to Numpy array for plotting
    if torch.is_tensor(adj_tensor):
        adj_matrix_np = adj_tensor.numpy()
    else:
        adj_matrix_np = adj_tensor

    plt.figure(figsize=(12, 10))

    # Create Heatmap
    # annot=True shows the actual weight values in the cells
    # cmap="viridis" or "Blues" are good color schemes
    sns.heatmap(adj_matrix_np,
                annot=True,
                fmt=".2f",
                cmap="viridis",
                xticklabels=station_labels,
                yticklabels=station_labels,
                cbar_kws={'label': 'Influence Weight'})

    plt.title("Spatial Graph Connections (Normalized Adjacency Matrix)")
    plt.xlabel("Target District (Influenced By)")
    plt.ylabel("Source District (Influencing)")
    plt.xticks(rotation=45)
    plt.show()

# --- Run Visualization ---
# Use the 'keep_stations' list from your earlier code as labels
visualize_adjacency_matrix(adj_matrix, keep_stations)

1. The Strong Diagonal (The "Self-Loop")
Observation: The diagonal line (top-left to bottom-right) is bright yellow/green, with values like 0.58, 0.62, 0.53.

Why this matters: This tells the model: "The most important predictor for District 101's pollution in the next hour is District 101's own pollution right now." This is physically true and stabilizes the LSTM.

2. Distinct Clusters (Geographic Neighborhoods)
Observation: Look at the bottom right corner (Districts 119, 120, 121, 122). They form a "block" of colors.

Why this matters: This indicates these districts are physically close to each other.

District 121 is heavily influenced by District 120 (weight 0.21) and itself (weight 0.62).

It effectively ignores distant districts like 101 or 102 (weight 0.00).

This proves your dynamic sigma calculation successfully identified local neighborhoods.

3. True Sparsity (The "Dark" Areas)
Observation: Large portions of the matrix are dark purple (0.00).

Why this matters: In your previous attempt, District 101 was influenced by District 122. In reality, wind doesn't teleport pollution instantly across the whole city.

By forcing these weights to zero, you have removed noise. The model will no longer try to find fake patterns between unrelated districts.

Conclusion
You have successfully moved from a "Global Average" model (the first image) to a true Spatio-Temporal Graph (the second image).

In [ ]:
class TGCN(nn.Module):
    def __init__(self, num_nodes, num_features, hidden_dim, output_dim, adj_matrix):
        super(TGCN, self).__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim
        self.adj = adj_matrix

        self.gcn_weight = nn.Parameter(torch.FloatTensor(num_features, hidden_dim))
        self.gcn_bias = nn.Parameter(torch.FloatTensor(hidden_dim))

        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self._init_weights()

    def _init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        self.gcn_weight.data.uniform_(-stdv, stdv)
        self.gcn_bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        batch_size, seq_len, num_nodes, _ = x.size()
        x_reshaped = x.view(-1, num_nodes, x.size(3))
        ax = torch.matmul(self.adj, x_reshaped)
        gcn_out = torch.relu(torch.matmul(ax, self.gcn_weight) + self.gcn_bias)
        gcn_out = gcn_out.view(batch_size, seq_len, num_nodes, self.hidden_dim)
        lstm_input = gcn_out.permute(0, 2, 1, 3).contiguous().view(batch_size * num_nodes, seq_len, self.hidden_dim)
        lstm_out, _ = self.lstm(lstm_input)
        out = self.fc(lstm_out[:, -1, :])
        return out.view(batch_size, num_nodes, -1)

In [ ]:
model = TGCN(NUM_NODES, NUM_FEATURES, 64, TARGET_DIM, adj_matrix)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
# Check shapes
print(f"Original train_y shape: {train_y.shape}")

# If train_y is (Batch, 6, 13), we must swap the last two dimensions to get (Batch, 13, 6)
if train_y.shape[-1] == 13 and train_y.shape[-2] == 6:
    print("Fixing dimensions...")
    train_y = train_y.permute(0, 2, 1)
    train_x = train_x  # x usually doesn't need changing if it's (Batch, Seq, Nodes, Features)

print(f"Corrected train_y shape: {train_y.shape}")

In [ ]:

# Create DataLoader for Batching
total_samples = len(train_x)
split_idx = int(total_samples * 0.8)

# Train on the first 80%
x_train_tensor = train_x[:split_idx]
y_train_tensor = train_y[:split_idx]

# Test on the last 20% (Future data)
x_test_tensor = train_x[split_idx:]
y_test_tensor = train_y[split_idx:]

print(f"Training Samples: {len(x_train_tensor)}")
print(f"Testing Samples:  {len(x_test_tensor)}")

# Train Loader: Shuffle=True is GOOD here (helps the model learn robustly)
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Test Loader: Shuffle=FALSE is CRITICAL here
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
print("Starting Training Loop...")

model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        predictions = model(batch_x)

        # DEBUG: Print shapes once to verify
        if epoch == 0 and total_loss == 0:
            print(f"Predictions shape: {predictions.shape}") # Should be [32, 13, 6]
            print(f"Targets shape:     {batch_y.shape}")     # Should be [32, 13, 6]

        loss = criterion(predictions, batch_y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")


print("Training Complete.")

In [ ]:
import torch

# model is your PyTorch model instance
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
# 1. Recreate the model architecture
model = TGCN(NUM_NODES, NUM_FEATURES, 64, TARGET_DIM, adj_matrix)

# 2. Load the saved weights
model.load_state_dict(torch.load('model_weights.pth'))
# model.eval()  # set to evaluation mode

In [ ]:
model.eval()
all_preds = []
all_targets = []

# 1. Collect Predictions
with torch.no_grad():
    for inputs, targets in test_loader:
        preds = model(inputs)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

#new: Shape is now (Total_Samples, 13, 6)
predictions = np.concatenate(all_preds, axis=0)
actuals = np.concatenate(all_targets, axis=0)

In [ ]:
# Helper function to inverse transform multi-feature output
def inverse_transform_multifeature(data_3d, scalers, target_cols):
    """
    data_3d shape: (Time, Nodes, Features)
    """
    samples, nodes, feats = data_3d.shape
    output = np.zeros_like(data_3d)

    for f_idx, feature_name in enumerate(target_cols):
        # 1. Extract the specific feature slice (Time, Nodes)
        slice_data = data_3d[:, :, f_idx]

        # 2. Reshape to (Time*Nodes, 1) for the scaler
        # slice_flat = slice_data.reshape(-1, 1)

        # 3. Inverse transform just this feature
        # Note: Scaler expects (N, 13) usually if trained that way,
        # but here we reshape to fit the scaler's logic.
        # IF your scalers were fit on shape (-1, 1), do this:
        slice_inv = scalers[feature_name].inverse_transform(slice_data)

        # 4. Reshape back and store
        output[:, :, f_idx] = slice_inv

    return output

In [ ]:
print("Inverse transforming predictions...")
pred_inv = inverse_transform_multifeature(predictions, scalers, TARGET_COLS)
actual_inv = inverse_transform_multifeature(actuals, scalers, TARGET_COLS)

In [ ]:
# --- EXAMPLE: Print Results for Station 101, First Test Sample ---
station_idx = 0
time_idx = 0

print(f"\nPrediction for Station {station_idx} at T+1:")
print(f"{'Feature':<10} | {'Pred':<10} | {'Actual':<10}")
print("-" * 35)

for i, feat in enumerate(TARGET_COLS):
    p_val = pred_inv[time_idx, station_idx, i]
    a_val = actual_inv[time_idx, station_idx, i]
    print(f"{feat:<10} | {p_val:.4f}     | {a_val:.4f}")

In [ ]:
 # --- Plot 1: Overall Performance (First District as example) ---
plt.figure(figsize=(12, 6))
plt.plot(actuals[:200, 0, 0], label='Actual PM2.5 (District 0)', color='black', alpha=0.7)
plt.plot(predictions[:200, 0, 0], label='Predicted PM2.5', color='cyan', linestyle='--')
plt.title("Forecasting: District 0 (First 200 Hours)")
plt.legend()
plt.show()



In [ ]:
# --- Plot 2: Error Distribution by District ---
# Calculate MAE for each district separately
mae_per_district = np.mean(np.abs(predictions - actuals), axis=0).flatten()

plt.figure(figsize=(10, 5))
sns.barplot(x=list(range(13)), y=mae_per_district, palette="viridis")
plt.xlabel("District ID")
plt.ylabel("Mean Absolute Error (PM2.5)")
plt.title("Model Error per District")
plt.show()

manual input

In [ ]:
def create_template_csv():
    dates = pd.date_range(end='2023-12-05 13:00', periods=24, freq='H')
    stations = [101, 102, 105, 106, 107, 109, 111, 112, 113, 119, 120, 121, 122]

    rows = []
    for d in dates:
        for s in stations:
            rows.append({
                'Measurement date': d,
                'Station code': s,
                'SO2': 0, 'NO2': 0, 'O3': 0, 'CO': 0, 'PM10': 0, 'PM2.5': 0
            })

    pd.DataFrame(rows).to_csv('prediction_input_template.csv', index=False)
    print("Template 'prediction_input_template.csv' created. Fill it with data.")


In [ ]:
create_template_csv()

In [ ]:
adj_matrix = torch.load("/content/adj_matrix.pt")

In [ ]:
# 1. Recreate the model architecture
model = TGCN(NUM_NODES, NUM_FEATURES, 64, TARGET_DIM, adj_matrix)

# 2. Load the saved weights
model.load_state_dict(torch.load('/content/model_weights.pth'))

In [ ]:
import joblib
scalers = joblib.load("/content/seoul_scalers.pkl")

In [ ]:
def predict_from_csv(csv_path, model, scalers):
    """
    Reads a CSV, processes the last 24 hours of data, and forecasts PM2.5.
    """
    # 1. Load Data
    input_df = pd.read_csv(csv_path)

    # 2. Preprocessing (Must match training logic exactly)
    input_df['Measurement date'] = pd.to_datetime(input_df['Measurement date'])

    # Filter for the 13 specific stations used in training
    keep_stations = [101, 102, 105, 106, 107, 109, 111, 112, 113, 119, 120, 121, 122]
    input_df = input_df[input_df['Station code'].isin(keep_stations)]

    # Sort to ensure node alignment (Time -> Station)
    input_df = input_df.sort_values(by=['Measurement date', 'Station code'])

    # Time Encoding
    input_df['hour'] = input_df['Measurement date'].dt.hour
    input_df['month'] = input_df['Measurement date'].dt.month
    input_df['hour_sin'] = np.sin(2 * np.pi * input_df['hour'] / 24.0)
    input_df['hour_cos'] = np.cos(2 * np.pi * input_df['hour'] / 24.0)
    input_df['month_sin'] = np.sin(2 * np.pi * input_df['month'] / 12.0)
    input_df['month_cos'] = np.cos(2 * np.pi * input_df['month'] / 12.0)

    # 3. Check Data Sufficiency
    # We need exactly 24 hours for 13 stations = 312 rows
    # We take the LAST 24 hours available in the CSV
    timestamps = input_df['Measurement date'].unique()
    if len(timestamps) < 24:
        raise ValueError(f"CSV only has {len(timestamps)} hours of data. Model requires 24 hours history.")

    last_24_hours = timestamps[-24:]
    input_df = input_df[input_df['Measurement date'].isin(last_24_hours)]

    # 4. Prepare Features (Normalize using TRAINED scalers)
    feature_cols = ['NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM2.5',
                    'hour_sin', 'hour_cos', 'month_sin', 'month_cos']

    processed_features = []

    for feat in feature_cols:
        # Reshape to (24 hours, 13 stations)
        values = input_df[feat].values.reshape(24, 13)

        # Handle Missing Values (Interpolate just like training)
        val_df = pd.DataFrame(values)
        val_df = val_df.interpolate(method='linear', limit_direction='both').bfill().ffill()

        # Scale
        if 'sin' not in feat and 'cos' not in feat:
            # CRITICAL: Use .transform(), do NOT use .fit_transform()
            if feat in scalers:
                values_norm = scalers[feat].transform(val_df.values)
            else:
                print(f"Warning: Scaler for {feat} not found. Using raw values.")
                values_norm = val_df.values
        else:
            values_norm = val_df.values

        processed_features.append(values_norm)

    # Stack to get shape (24, 13, 10)
    input_seq = np.stack(processed_features, axis=-1)

    # Add Batch Dimension: (1, 24, 13, 10)
    input_tensor = torch.FloatTensor(input_seq).unsqueeze(0)

    # 5. Prediction
    model.eval()
    with torch.no_grad():
        # Output shape: (1, 13, 6)
        prediction = model(input_tensor)

    # 6. Inverse Scale PM2.5
    # PM2.5 is at index 5 in the target list
    # prediction shape is (1, 13, 6) -> We want (13,) values for PM2.5
    pm25_values = prediction[0, :, 5].numpy()

    # CRITICAL FIX: Reshape to (1, 13)
    # The scaler expects (n_samples, 13_stations), so we pretend this is 1 sample of 13 stations
    pm25_input_for_scaler = pm25_values.reshape(1, 13)

    # Inverse transform
    pm25_actual = scalers['PM2.5'].inverse_transform(pm25_input_for_scaler)

    # Flatten back to a list of 13 values for the dataframe
    pm25_final = pm25_actual.flatten()

    # 7. Format Results
    results = pd.DataFrame({
        'Station Code': keep_stations,
        'Predicted PM2.5': pm25_final
    })

    return results


In [ ]:

# --- Usage Example ---
# Assuming 'model' is trained and 'scalers' is populated from your previous run
try:
    # Use a CSV that has at least the last 24 hours of data
    forecast_df = predict_from_csv('/content/aqi_data.csv', model, scalers)

    print("\n--- Forecast for Next Hour ---")
    print(forecast_df)

    # Optional: Save to CSV
    # forecast_df.to_csv('forecast_results.csv', index=False)

except Exception as e:
    print(f"Error during prediction: {e}")

Prediction for hour following: 2023-12-05 13:00:00 means it forecasts the value for the next hour (which is 14:00), since our model was trained to predict the next hour.

can change the code into
```
# Calculate the actual future time
future_time = pd.to_datetime(time_point) + pd.Timedelta(hours=1)
print(f"Prediction target time: {future_time}")
```

